In [1]:
# Import necessary libraries
import googlemaps
import datetime
import pandas as pd

In [2]:
# Set up the Google Maps API
API_KEY = 'YOUR_GOOGLE_MAPS_API_KEY'
gmaps = googlemaps.Client(key=API_KEY)

### Function to estimate travel time between two points

In [3]:
def estimate_travel_time(origin, destination, time_departure):
    # Get the travel time from Google Maps API
    directions_result = gmaps.directions(origin, destination, mode='driving', departure_time=time_departure)
    travel_time_seconds = directions_result[0]['legs'][0]['duration']['value']
    
    return travel_time_seconds / 60  # Convert to minutes


### Example of a rider dictionary

In [4]:
# Example of a rider dictionary
rider_382534230 = {
    "account_ID": None,
    "pickup_by": None, #slightly higher than waiting time that is given to rider, this value is used for constraint
    "pickup_time": None,
    "arrive_by": None, # a reasonable max detour time has been added to the ETA, this value is used for constraint
    "ETA_direct": None,
    "pickup_location": None,
    "dropoff_location": None,
    "completion_time": None
}


### Functions to get potential riders, update rider status, and update the current rider

The following functions are not directly related to the shared riding algorithm, so they will be implemented as separate modules:

In [5]:
def get_potential_rider(driver_location, radius):
    # this function filters potential riders that are within range 
    pass
     

def update_rider(rider, status):
    # automatically triggers when a rider initiates a request, gets in the car, and completes the ride
    # when a rider gets in the car, update the pickup_time and ETA_direct
    if status == 'picked up':
        rider['pickup_time'] = datetime.datetime.now()
        rider['ETA_direct'] = datetime.datetime.now() + estimate_travel_time(rider['pickup_location'], rider['dropoff_location'], 'now')
    
    # other codes
    # ...

def update_current():
    #codes to update the current rider for each driver/vehicle  
    # if 2 riders are in the car, return the one who connected first
    # if 1 rider has been dropped off, the next rider in the queue will be returned
    return current, number_onboard

## Function to generate path options based on the current situation

#### Assumption: max capacity of 2 passengers
Let's start by considering a simple case where the capacity for shared rides is 2 passengers. If you're riding alone, you may be matched with another rider going in the same direction, resulting in a shared ride with up to 2 passengers. If you're riding with someone else, you will not be matched with additional riders, as you have already reached the maximum capacity.

In this case, let's consider 2 passengers A and B. Assume A connects with the driver first. Next, we need to find a passenger B who might be eligible to share the ride. Say A wants to go from point A1 to A2, and B from point B1 to B2. X is the current location of the driver.

**Scenario 1**: If the driver has already picked up A, then point A1 has been visited. The possible paths for a potential shared ride will be either:

	A1(visited) -> X -> B1 -> B2 -> A2, or
	A1(visited) -> X -> B1 -> A2 -> B2

**Scenerio 2**: If the driver is on the way to pick up A, then the possible paths for a potential shared ride will be one of the following:

	A1 -> B1 -> B2 -> A2
	A1 -> B1 -> A2 -> B2
	B1 -> A1 -> B2 -> A2 (there is chance that B's pickup location could be closer to A's)
	B1 -> A1 -> A2 -> B2
    
Next, let's define a function to generate path options.

**Note**: I will refer to A as `current`, and B as `potential`

In [6]:
def get_path_options(potential):
    
    path_options = []    
# -----SCENERIO 1: IF THERE IS NO ONE IN THE CAR AT THE MOMENT(ON THE WAY TO PICK UP THE current RIDER) ----
    
    if number_onboard == 0:
        # Define path options with different possible node ordering
        path_a = [driver_location, current['pickup_location'], potential['pickup_location'], current['dropoff_location'], potential['dropoff_location']]
        path_b = [driver_location, current['pickup_location'], potential['pickup_location'], potential['dropoff_location'], current['dropoff_location']]
        path_c = [driver_location, potential['pickup_location'], current['pickup_location'], current['dropoff_location'], potential['dropoff_location']]
        path_d = [driver_location, potential['pickup_location'], current['pickup_location'], potential['dropoff_location'], current['dropoff_location']]
        
        path_options = [path_a, path_b, path_c, path_d]


# -----SCENERIO 2: IF the first current rider is in car already ----
    
    elif number_onboard == 1:
        path_e = [driver_location, potential['pickup_location'], current['dropoff_location'], potential['dropoff_location']]
        path_f = [driver_location, potential['pickup_location'], potential['dropoff_location'], current['dropoff_location']]

        path_options = [path_e, path_f]

# -----SCENERIO 3: when 2 riders are in the car, the capacity has been reached(assumption), function will not be called

    return path_options

### Function to determine if a shared ride is acceptable based on constraints

- **estimated time of pickup should be earlier than `pickup_by` time for both riders** <Br>
(`pickup_by` will be generated in another module with an acceptable waiting time added, which serves as a rough constraint, it could be a flat number say 10 minutes, or it could also be optimized depending on each user's behavior analysis)
    
    
- **estimated time of arrival should be earlier than `arrive_by` time for both riders** <Br>
(similar to above, `arrive_by` will be generated in another module with an acceptable detour time added, which serves as a rough constraint, it could be some optimized range(say 10-20 minutes) depending on travel distance and each user's behavior analysis)  
    
    
- **detour time for each rider is less than max detour time** <Br>
(we can set a max detour time as a constraint for each rider given their estimated travel distance and time for direct trip without sharing ride. To simplify the logics, I will start by using a flat max detour time ) 

#### Set constraint parameters

In [7]:
# Maximum detour time allowed
max_detour_time = 15


The `is_shared_ride_acceptable` function below evaluates if a shared ride is acceptable for both current and potential passengers, considering the maximum detour time and the number of passengers on board. The function calculates real-time ETA and returns a DataFrame containing feasible path options.

Inputs:
- current: The current passenger's ride information
- potential: The potential passenger's ride information
- max_detour_time: The maximum detour time allowed for each passenger
- number_onboard: The number of passengers currently on board

The function performs the following steps:

1. Generate path options for the shared ride using the `get_path_options` function.
2. Loop over all path options and calculate the travel time for each leg in the path.
3. Determine pickup and dropoff nodes for both current and potential passengers.
4. Calculate the estimated time of pickup (ETP) and estimated time of arrival (ETA) for each ride.
5. Calculate the estimated time of direct travel for both passengers.
6. Calculate the detour time for each passenger.
7. Append the information for each path option to the `shared_ride_schedule` DataFrame.
8. Apply constraints on pickup time, arrival time, and detour time.
9. Filter feasible path options and return a DataFrame containing the feasible paths.



In [8]:
def is_shared_ride_acceptable(current, potential, max_detour_time, number_onboard):
    # this function will calculate real time ETA and decide whether a shared ride is accceptable given the current

    path_options = get_path_options(current, potential)
    
    # Create an empty dataframe for share
    shared_ride_schedule = pd.DataFrame(columns=['path', 
                                                'ETP_current', 'ETA_current', 'detour_time_current', 
                                                'ETP_potential', 'ETA_potential', 'detour_time_potential', 
                                                'total_travel_time'])
        
    # Loop over all paths
    for i, path in enumerate(path_options):
        # Initialize time_departure to current time
        time_departure = datetime.datetime.now()
        
        # Calculate total travel time
        total_travel_time = 0
        time_schedule = [time_departure]
        for j in range(len(path)-1):
            leg_travel_time = estimate_travel_time(path[j], path[j+1], time_departure)
            total_travel_time += leg_travel_time
            
            # Update time_departure for the next leg
            time_departure += datetime.timedelta(minutes=leg_travel_time)
            
            # record the departure time of each leg
            time_schedule.extend(time_departure)          

        # Find pickup and dropoff node for each ride
        if number_onboard == 0: 
            current_start_node = path.index(current['pickup_location']) # only when the first rider hasn't been picked up      

        current_end_node = path.index(current['dropoff_location'])       
                
        potential_start_node = path.index(current['pickup_location'])       
        potential_end_node = path.index(potential['dropoff_location'])

        # Calculate estimated time of pickup and arrival for each ride, which is also the departure time from its leg
        if number_onboard == 0: 
            ETP_current = current['pickup_time'] # if the first ride is in car, then his pickup_time has been updated already
        else:
            ETP_current = time_schedule[current_start_node]
            
        ETA_current = time_schedule[current_end_node]
        
        ETP_potential = time_schedule[potential_start_node]
        ETA_potential = time_schedule[potential_end_node]
        
        # Calculate estimated time of DIRECT travel
        if number_onboard == 0: 
            ETA_direct_current = current['ETA_direct'] # if the first ride is in car, then his direct ETA has been assessed and updated into his dictionary
        else:
            ETA_direct_current = ETP_current + estimate_travel_time(current['pickup_location'], current['dropoff_location'], ETP_current)

        ETA_direct_potential = ETP_potential + estimate_travel_time(potential['pickup_location'], potential['dropoff_location'], ETP_potential)
        
        # Calculate detour time in minutes
        detour_time_current = (ETA_current - ETA_direct_current).total_seconds() /60
        detour_time_potential = (ETA_potential - ETA_direct_potential).total_seconds() /60
        
        # Append info of each path option to shared_ride_schedule dataframe as a single row
        shared_ride_schedule.loc[i] = [path, \
                                      ETP_current, ETA_current, detour_time_current, \
                                      ETP_potential, ETA_potential, detour_time_potential, \
                                      total_travel_time]


    # set constraints: 
    # 1. estimated time of pickup should be earlier than "pickup_by" time for both riders
    const_pickup_by = (shared_ride_schedule['ETP_current'] <= current['pickup_by']) \
                        & (shared_ride_schedule['ETP_potential'] <= potential['pickup_by'])

    # 2. estimated time of arrival should be earlier than "arrive_by" time for both riders
    const_arrive_by = (shared_ride_schedule['ETA_current'] <= current['arrive_by']) \
                        & (shared_ride_schedule['ETA_potential'] <= potential['arrive_by'])

    # 3. detour time for each rider less than max detour time
    const_detour_time = (shared_ride_schedule['detour_time_current'] <= max_detour_time) \
                        & (shared_ride_schedule['detour_time_potential'] <= max_detour_time)

    # filter the feasible path options    
    feasible_paths = shared_ride_schedule[const_pickup_by & const_arrive_by & const_detour_time]

    
    if feasible_paths.shape[0] > 0:
        feasible_paths['account_ID'] = potential['account_ID'] # potential['account_ID'] is a string, it will create a column of that string

    return feasible_paths

## Main function: to match riders based on the most optimal shared ride

This function matches the current rider with a potential rider for a shared ride based on the least total travel time. It first updates the current primary rider and the number of riders in the car. Then, it finds potential riders within a certain radius of the driver's location. For each potential rider, the function checks if a shared ride is acceptable based on the constraints set in the `is_shared_ride_acceptable` function. All feasible paths for all potential riders are stored in a DataFrame. Finally, the function sorts the feasible paths by total travel time and outputs the best match and the corresponding route details.

Here's a breakdown of the steps in the function:

1. Update the current primary rider and the number of riders in the car using the `update_current()` function.
2. Get potential riders within range using the `get_potential_riders(driver_location, radius)` function.
3. Create an empty DataFrame to store all feasible paths for all potential riders.
4. Iterate through each potential rider and get the feasible paths using the `is_shared_ride_acceptable()` function. Add these paths to the DataFrame.
5. Sort the DataFrame by total travel time.
6. Extract the information about the best match and the corresponding route details, such as detour time and estimated time of arrival for both riders.
7. Print the details of the matched rider, the best route, and the travel times for both riders.


In [9]:

def match_rider(current):
    
    # update current primary rider and number of riders in the car
    current, number_onboard = update_current()

    # Get potential riders within range
    potential_riders = get_potential_riders(driver_location, radius)

    # Create an empty dataframe to store all feasible paths for all potential riders
    potential_feasible_paths = pd.DataFrame()
    
    # Iterate through each potential rider
    for potential in potential_riders:
        # Get feasible paths 
        feasible_paths = is_shared_ride_acceptable(current, potential, max_detour_time, number_onboard)
        
        # Add to the dataframe
        if df.shape[0] > 0:
            potential_feasible_paths = potential_feasible_paths.append(feasible_paths, ignore_index=True)
    
    # Sort by total_travel_time     
    potential_sorted = potential_feasible_paths.sort_values(by='total_travel_time').reset_index()
    # (Note: we can also choose other optimization method such as a more balanced detour time for each rider. for example,
    # how would we choose between detour time combination (0 minute, 15 minute) and (10 min, 9 min)?, we can also design A/B testing for it)
    
    current_rider_ID = current['account_ID']
    match_rider_ID = potential_sorted['account_ID'][0]  
    best_path = potential_sorted['path'][0],
    detour_time_current = potential_sorted['detour_time_current'][0],
    detour_time_match = potential_sorted['detour_time_potential'][0],
    ETA_current = potential_sorted['ETA_current'][0],
    ETA_match = potential_sorted['ETA_potential'][0]
        
    print('Share ride with: ', match_rider_ID)            
    print('The best route with least total travel time is: ', best_path)
    print(f'Detour time for {current_rider_ID}: ', detour_time_current)
    print(f'Detour time for {match_rider_ID}: ', detour_time_match)
    print(f'Estimated time of arrival for {current_rider_ID}: ', detour_time_current)
    print(f'Estimated time of arrival for {match_rider_ID}: ', detour_time_match)
        
        